# 11411/611 -NLP (S23)
# Asssignment 3: Naive Bayes

Classifiers are helpful in distinguish texts from different categories. They are vey useful in numerous use cases.

In this assignment you will build a Naive Bayes Classifier that will distinguish 6 different languages namely Hausa, Indonesisan, Manobo, Tagalog, Swahili and Nahuatl

You are required to program a Naive Bayes Classifier in this HW.

Submission Guidelines
Deadline: 

Programming:

This notebook contains helpful test cases and additional information about the programming part of the HW. However, you are only required to submit naivebayes.py on Gradescope.
We recommended that you first code in the notebook and then copy the corresponding methods/classes to naivebayes.py

In [967]:
from collections import defaultdict, Counter
from operator import itemgetter
from math import log
from typing import DefaultDict
import sys
import io

In [968]:
class NaiveBayes():

    def extract_ngrams(self,x: str, n=2) -> "list[str]":
        """
        Train a Naive-Bayes model

        :param x: The document which needs to be decomposed into ngrams.
        :para n: the order of ngrams.
        :return: list of ngrams
        """
        ###TODO###
        #extract character ngrams
        n_grams = []
        
        for i in range(0, len(x)):
            n_grams.append(x[i : i + n])
        
        n_grams = n_grams[:len(n_grams) - n + 1]
        
        return n_grams

    def smoothed_log_likelihood(self, w: str, c: str, k: int, count: 'DefaultDict[str, Counter]', vocab: "set[str]") -> float:
        """
        :param w: word in the vocab
        :para c: class label
        :param k: The value added to the numerator and denominator to smooth likelihoods
        :para count: Dictionary containing the count of label c occuring with an ngram w
        :param vocab: the vocabulary for the model
        :type b: int
        :return: the log likelihood value after smoothening
        """
        ###TODO###
        #apply smoothing
        summation_of_vocab_count = 0
        word_count = count[w][c]
        
        for word in count.keys():
            summation_of_vocab_count += count[word][c] + k
        log_likelihood = log((word_count + k)/(summation_of_vocab_count))

        return log_likelihood

    def train_nb(self, docs: "list[tuple[str, str]]", k: int = 1, n: int = 2) -> "tuple[dict[str, float], DefaultDict[str, DefaultDict[str, float]], set[str], set[str]]":
        ###TODO###
        """
        Train a Naive-Bayes model

        :param docs: The documents, each associated with a clas label (document, label)
        :type docs: list[tuple[str, str]]
        :param k: The value added to the numerator and denominator to smooth likelihoods
        :type k: int
        :para n: the order of ngrams.
        :type b: int
        :return: The log priors, log likelihoods, the classes, and the vocabulary for the model at a tuple
        :rtype: tuple[dict[str, float], DefaultDict[str, DefaultDict[str, float]], set[str], set[str]]
        """
        num_of_docs = len(docs)
        log_priors = {}

        for doc in docs:
            doc_label = doc[0]

            if doc_label in log_priors:
                log_priors[doc_label] += 1
            else:
                log_priors[doc_label] = 1

            
        labels = log_priors.keys()
        big_doc = {}

        for label in labels:
            log_priors[label] = log(log_priors[label]/num_of_docs)
            big_doc[label] = []

        vocabulary = {}
        vocab_frequency = {}
        for doc in docs:
            for vocab in self.extract_ngrams(doc[1], n):
                vocabulary[vocab] = True
                if vocab not in vocab_frequency:
                    vocab_frequency[vocab] = {}
                    
                    for label in labels:
                        if label not in vocab_frequency[vocab]:
                            vocab_frequency[vocab][label] = 0
                    vocab_frequency[vocab][doc[0]] = 1
                else:
                    vocab_frequency[vocab][doc[0]] += 1

        vocabulary = vocabulary.keys()
        print(vocab_frequency)
        print(docs)
        
        log_likelihoods = {}
        for word in vocabulary:
            for label in labels:
                log_likelihood = self.smoothed_log_likelihood(word, label, k, vocab_frequency, vocabulary)
                
                if word not in log_likelihoods:
                    log_likelihoods[word] = {label:log_likelihood}
                else:
                    log_likelihoods[word][label] = log_likelihood
            
        return log_priors, log_likelihoods, labels, vocabulary

    def classify(self, testdoc: str, log_prior: "dict[str, float]", log_likelihood: "DefaultDict[str, DefaultDict[str, float]]", classes: "set[str]", vocab: "set[str]", k: int=1, n: int=2) -> str:
        ###TODO###
        """Given a trained NB model (log_prior, log_likelihood, classes, and vocab), returns the most likely label for the input document.

        :param textdoc str: The test document.
        :param log_prior dict[str, float]: The log priors of each category. Categories are keys and log priors are values.
        :param log_likelihood DefaultDict[str, DefaultDict[str, float]]: The log likelihoods for each combination of word/ngram and class.
        :param classes set[str]: The set of class labels (as strings).
        :param vocab set[str]: The set of words/negrams in the vocabulary.
        :param k int: the value added in smoothing.
        "param n int: the order of ngrams.
        :return: The best label for `testdoc` in light of the model.
        :rtype: str
        """

        ##TODO
        # Extract a set of ngrams from `testdoc`
        doc = self.extract_ngrams(testdoc)
        ##TODO
        # Initialize the sums for each class. These will be the "scores" based on which class will be assigned.
        class_sum = {}

        frequency = {}
        total_frequency = 0

        for vocab in doc:
            count = testdoc.count(vocab)
            if vocab not in frequency:
                frequency[vocab] = count + k
                total_frequency += count + k
                
        ##TODO
        # Iterate over the classes, computing `class_sum` for each
        for c in classes:
            ##TODO
            # Initialize `class_sum` with the log prior for the class
            ##TODO
            # Then add the likelihood for each in-vocabulary word/ngram in the document
            class_sum[c] = 0
                    
            for vocab in doc:
                try:
                    class_sum[c] += log_likelihood[vocab][c]
                except:    
                    class_sum[c] += log(frequency[vocab]/(total_frequency - frequency[vocab]))

        best_class = list(class_sum.keys())[0]
        best_class_value = class_sum[best_class]

        for a_class in class_sum.keys():
            if best_class_value <= class_sum[a_class]:
                best_class = a_class
                best_class_value = class_sum[a_class]
        
        return best_class
        
            
        

    def precision(self,tp: "dict[str, int]", fp: "dict[str, int]") -> float:
        return tp / (tp + fp)

    def recall(self,tp: "dict[str, int]", fn: "dict[str, int]") -> float:
        return tp / (tp + fn)

    def micro_precision(self, tp: "dict[str, int]", fp: "dict[str, int]") -> float:
        tp_sum = sum(tp.values())
        fp_sum = sum(fp.values())
        return tp_sum / (tp_sum + fp_sum)

    def micro_recall(self, tp: "dict[str, int]", fn: "dict[str, int]") -> float:
        tp_sum = sum(tp.values())
        fn_sum = sum(fn.values())
        return tp_sum / (tp_sum + fn_sum)

    def micro_f1(self, tp: "dict[str, int]", fp: "dict[str, int]", fn: "dict[str, int]") -> float:
        mp = self.micro_precision(tp, fp)
        mr = self.micro_recall(tp, fn)
        return 2 * (mp * mr) / (mp + mr)

    def macro_precision(self, tp: "dict[str, int]", fp: "dict[str, int]") -> float:
        n = len(tp)
        return (1 / n) * sum([self.precision(tp[c], fp[c]) for c in tp.keys()])

    def macro_recall(self, tp: "dict[str, int]", fn: "dict[str, int]") -> float:
        n = len(tp)
        return (1 / n) * sum([self.recall(tp[c], fn[c]) for c in tp.keys()])

    def macro_f1(self, tp: "dict[str, int]", fp: "dict[str, int]", fn: "dict[str, int]") -> float:
        n = len(tp)
        return 2 * (self.macro_precision(tp, fp) * self.macro_recall(tp, fn)) / (self.macro_precision(tp, fp) + self.macro_recall(tp, fn))

    def evaluate(self, train: "list[tuple[str, str]]", eval: "list[tuple[str, str]]", n: int=2):
        log_prior, log_likelihood, classes, vocab = self.train_nb(train, n = n)
        # Initialize dictionaries for true positives, false positives, and false negatives
        tp, fp, fn = defaultdict(int), defaultdict(int), defaultdict(int)
        confusion = defaultdict(lambda: defaultdict(int))
        for c_ref, doc in eval:
            c_hyp = self.classify(doc, log_prior, log_likelihood, classes, vocab, n = n)
            confusion[c_ref][c_hyp] += 1
            if c_ref == c_hyp:
                tp[c_ref] += 1
            else:
                fn[c_ref] += 1
                fp[c_hyp] += 1

        print(f'Macro-averaged precision:\t{self.macro_precision(tp, fp)}')
        print(f'Macro-averaged recall:\t{self.macro_recall(tp, fn)}')
        print(f'Macro-averaged F1:\t{self.macro_f1(tp, fp, fn)}')
        print(f'Micro-averaged precision:\t{self.micro_precision(tp, fp)}')
        print(f'Micro-averaged recall:\t{self.micro_recall(tp, fn)}')
        print(f'Micro-averaged F1:\t{self.micro_f1(tp, fp, fn)}')

        return self.macro_precision(tp, fp), self.macro_recall(tp, fn), self.macro_f1(tp, fp, fn), self.micro_precision(tp, fp), self.micro_recall(tp, fn), self.micro_f1(tp, fp, fn)


In [969]:
with open('train_small.tsv',  encoding='utf-8') as f:
    train = [tuple(l.split('\t')) for l in f]
    
with open('test_small.tsv',  encoding='utf-8') as f:
    dev = [tuple(l.split('\t')) for l in f]

In [970]:
#Test Trained model
nb_model = NaiveBayes()

In [971]:
log_prior, log_likelihood, classes, vocab = nb_model.train_nb(train, n = 1)

{'Y': {'1': 2, '0': 2}, 'o': {'1': 7, '0': 4}, 'u': {'1': 3, '0': 3}, ' ': {'1': 12, '0': 10}, 'a': {'1': 6, '0': 7}, 'r': {'1': 5, '0': 5}, 'e': {'1': 9, '0': 5}, 't': {'1': 3, '0': 0}, 'h': {'1': 1, '0': 0}, 'b': {'1': 2, '0': 2}, 's': {'1': 2, '0': 2}, '\n': {'1': 2, '0': 2}, 'd': {'1': 3, '0': 4}, 'p': {'1': 3, '0': 2}, 'n': {'1': 1, '0': 3}, 'N': {'1': 1, '0': 0}, 'l': {'1': 3, '0': 0}, 'g': {'1': 1, '0': 0}}
[('1', 'You are the best\n'), ('0', 'You are a bad person\n'), ('0', 'You are bad and a rude person\n'), ('1', 'Not all bad people are rude\n'), ('1', 'You are a good person')]


In [972]:
#Test log prior
assert str(log_prior['1']) == '-0.5108256237659907'
assert str(log_prior['0']) == '-0.916290731874155'

In [973]:
#Test vocab
print(vocab)
assert vocab == {'\n', 'g', 'p', 'o', 'd', ' ', 'a', 'h', 'n', 's', 'b', 'u', 't', 'e', 'l', 'Y', 'r', 'N'}

dict_keys(['Y', 'o', 'u', ' ', 'a', 'r', 'e', 't', 'h', 'b', 's', '\n', 'd', 'p', 'n', 'N', 'l', 'g'])


In [974]:
#Test log likelihood
print(log_likelihood['g'])
assert log_likelihood['g'] == {'1': -3.7376696182833684, '0': -4.23410650459726}
assert log_likelihood['p'] == {'1': -3.044522437723423, '0': -3.1354942159291497}
assert log_likelihood['Y'] == {'1': -3.332204510175204, '0': -3.1354942159291497}
assert log_likelihood['N'] == {'1': -3.7376696182833684, '0': -4.23410650459726}
assert log_likelihood['a'] == {'1': -2.4849066497880004, '0': -2.1546649629174235}
assert log_likelihood['s'] == {'1': -3.332204510175204, '0': -3.1354942159291497}

{'1': -3.7376696182833684, '0': -4.23410650459726}


In [977]:

#Evaluation
map, mar, maf, mp, mr, mf = nb_model.evaluate(train, dev, n=1)

{'Yo': {'1': 2, '0': 2}, 'ou': {'1': 2, '0': 2}, 'u ': {'1': 2, '0': 2}, ' a': {'1': 5, '0': 5}, 'ar': {'1': 3, '0': 2}, 're': {'1': 3, '0': 2}, 'e ': {'1': 5, '0': 3}, ' t': {'1': 1, '0': 0}, 'th': {'1': 1, '0': 0}, 'he': {'1': 1, '0': 0}, ' b': {'1': 2, '0': 2}, 'be': {'1': 1, '0': 0}, 'es': {'1': 1, '0': 0}, 'st': {'1': 1, '0': 0}, 't\n': {'1': 1, '0': 0}, 'a ': {'1': 1, '0': 2}, 'ba': {'1': 1, '0': 2}, 'ad': {'1': 1, '0': 2}, 'd ': {'1': 2, '0': 3}, ' p': {'1': 2, '0': 2}, 'pe': {'1': 2, '0': 2}, 'er': {'1': 1, '0': 2}, 'rs': {'1': 1, '0': 2}, 'so': {'1': 1, '0': 2}, 'on': {'1': 1, '0': 2}, 'n\n': {'1': 0, '0': 2}, 'an': {'1': 0, '0': 1}, 'nd': {'1': 0, '0': 1}, ' r': {'1': 1, '0': 1}, 'ru': {'1': 1, '0': 1}, 'ud': {'1': 1, '0': 1}, 'de': {'1': 1, '0': 1}, 'No': {'1': 1, '0': 0}, 'ot': {'1': 1, '0': 0}, 't ': {'1': 1, '0': 0}, 'al': {'1': 1, '0': 0}, 'll': {'1': 1, '0': 0}, 'l ': {'1': 1, '0': 0}, 'eo': {'1': 1, '0': 0}, 'op': {'1': 1, '0': 0}, 'pl': {'1': 1, '0': 0}, 'le': {'1': 1

In [976]:
print(map, mar, maf, mp, mr, mf)

0.6 1.0 0.7499999999999999 0.6 0.6 0.6
